<a href="https://colab.research.google.com/github/NUMBART/Face-Emotion-Recognition-System/blob/master/FacialExpressionTrainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Imports**

In [0]:
import os
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

**Mounting Google Drive and changing Current Working Directory**

*Mounting Google Drive*

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


*Function to create directory tree and forced remount*

In [0]:
#drive.mount("/content/drive", force_remount=True)
#stackoverflow.com/questions/9727673/list-directory-tree-structure-in-python
def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print('{}{}'.format(subindent, f))
list_files('/content/drive/My Drive/dataset/face-expression-recognition-dataset')

*Setting CWD*

In [0]:
os.chdir('/content/drive/My Drive/dataset/face-expression-recognition-dataset')
cwd = os.getcwd()
files = os.listdir(cwd)  # Get all the files in that directory
print("Files in %r: %s" % (cwd, files))

**Setting Paths**

In [0]:
train_path = 'images/train'
#test_path = 'images/test'
validation_path = 'images/validation'

**Creating validation set**

In [0]:

source1 = train_path
dest11 = "Dataset/validation"
files = os.listdir(source1)
print(files)
import shutil
import numpy as np
#iterating through classes in "train" directory if "Dataset/validation" doesn't exist
if not os.path.exists(dest11):
    for f in files:
        source_class_path =  source1 + '/' + f
        if os.path.isdir(source_class_path):
            class1 = os.listdir(source_class_path)
            #iterating through images in each class
            for img in class1:
                #using a 15/85 split on validation to training data
                if np.random.rand(1) < 0.22:
                    source_img_path = source1 + '/'+ f + '/' + img
                    dest_img_path = dest11 + '/' + f + '/' + img
                    dest_class_path = dest11 + '/' + f
                    if os.path.isfile(source_img_path):
                        if(os.path.exists(dest_class_path)):
                            shutil.move(source_img_path, dest_img_path)
                        #creating validation and class directories to store images
                        else:
                            os.makedirs(dest_class_path)
                            shutil.move(source_img_path, dest_img_path)


'\nsource1 = train_path\ndest11 = "Dataset/validation"\nfiles = os.listdir(source1)\nprint(files)\nimport shutil\nimport numpy as np\n#iterating through classes in "train" directory if "Dataset/validation" doesn\'t exist\nif not os.path.exists(dest11):\n    for f in files:\n        source_class_path =  source1 + \'/\' + f\n        if os.path.isdir(source_class_path):\n            class1 = os.listdir(source_class_path)\n            #iterating through images in each class\n            for img in class1:\n                #using a 15/85 split on validation to training data\n                if np.random.rand(1) < 0.22:\n                    source_img_path = source1 + \'/\'+ f + \'/\' + img\n                    dest_img_path = dest11 + \'/\' + f + \'/\' + img\n                    dest_class_path = dest11 + \'/\' + f\n                    if os.path.isfile(source_img_path):\n                        if(os.path.exists(dest_class_path)):\n                            shutil.move(source_img_path, d

In [0]:
'''
files = os.listdir(dest11)
print(files)
list_files(cwd)
'''

'\nfiles = os.listdir(dest11)\nprint(files)\nlist_files(cwd)\n'

**Creating batches**

In [0]:
train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size = (64, 64), color_mode = 'grayscale', classes = ['angry', 'happy', 'sad', 'surprise'], batch_size = 64)
validation_batches = ImageDataGenerator().flow_from_directory(validation_path, target_size = (64, 64), color_mode = 'grayscale', classes = ['angry', 'happy', 'sad', 'surprise'], batch_size = 64)
#test_batches = ImageDataGenerator().flow_from_directory('Dataset', target_size = (64, 64), color_mode = 'grayscale', classes = ['test'], batch_size = 30, shuffle = 'False')

Found 19310 images belonging to 4 classes.
Found 4721 images belonging to 4 classes.


**Checking batches by plotting images**

In [0]:
# plots images with labels within jupyter notebook
#https://github.com/smileservices/keras_utils/blob/master/utils.py#L6
def plots(ims, figsize=(12,6), rows=1, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], interpolation=None if interp else 'none')

In [0]:
'''
imgs, labels  = next(train_batches)
print(next(train_batches))
'''

'\nimgs, labels  = next(train_batches)\nprint(next(train_batches))\n'

In [0]:
'''
plots(imgs, figsize = (60,50), titles = labels)
'''

'\nplots(imgs, figsize = (60,50), titles = labels)\n'

**Build and Train CNN**

In [0]:
model = Sequential()

model.add(Conv2D(24, kernel_size = 3, padding = 'same', activation = 'relu', input_shape = (64,64,1)))
model.add(BatchNormalization())
model.add(Conv2D(24, kernel_size = 3, padding = 'same', activation = 'relu'))
model.add(BatchNormalization())
model.add(Conv2D(24,kernel_size=5,strides=3,padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(48, kernel_size = 3, padding = 'same', activation = 'relu'))
model.add(BatchNormalization())
model.add(Conv2D(48, kernel_size = 3, padding = 'same', activation = 'relu'))
model.add(BatchNormalization())
model.add(Conv2D(48, kernel_size=5, strides=3, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(84, kernel_size = 3, padding = 'same', activation = 'relu'))
model.add(BatchNormalization())
model.add(Conv2D(84, kernel_size = 3, padding = 'same', activation = 'relu'))
model.add(BatchNormalization())
model.add(Conv2D(84, kernel_size=5, strides=2, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(96, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Dense(4, activation = 'softmax'))


In [0]:
model.summary()

In [0]:
model.compile(Adam(lr = 0.01), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [0]:
model.fit_generator(train_batches, steps_per_epoch = 150, validation_data = validation_batches, validation_steps = 36, epochs = 10, use_multiprocessing=True,
  workers=16, verbose = 2)

Epoch 1/10
 - 16s - loss: 0.2017 - acc: 0.9267 - val_loss: 0.8586 - val_acc: 0.7739
Epoch 2/10
 - 13s - loss: 0.2283 - acc: 0.9190 - val_loss: 0.9780 - val_acc: 0.7548
Epoch 3/10
 - 16s - loss: 0.2044 - acc: 0.9253 - val_loss: 0.8752 - val_acc: 0.7724
Epoch 4/10
 - 13s - loss: 0.2163 - acc: 0.9250 - val_loss: 0.9718 - val_acc: 0.7509
Epoch 5/10
 - 16s - loss: 0.1882 - acc: 0.9321 - val_loss: 0.9182 - val_acc: 0.7466
Epoch 6/10
 - 14s - loss: 0.2178 - acc: 0.9190 - val_loss: 0.8849 - val_acc: 0.7613
Epoch 7/10
 - 16s - loss: 0.1904 - acc: 0.9318 - val_loss: 0.8694 - val_acc: 0.7650
Epoch 8/10
 - 14s - loss: 0.2118 - acc: 0.9242 - val_loss: 0.8712 - val_acc: 0.7691
Epoch 9/10
 - 16s - loss: 0.1792 - acc: 0.9358 - val_loss: 0.9643 - val_acc: 0.7540
Epoch 10/10
 - 14s - loss: 0.1924 - acc: 0.9289 - val_loss: 0.8747 - val_acc: 0.7552


**Saving Model**

In [0]:
model.save('FacialExpressionRecogniser01.h5')

In [0]:
new_model = load_model('FacialExpressionRecogniser01.h5')

**Prediction**

In [0]:
'''
plot_imgs, plot_labels  = next(test_batches)
'''

'\nplot_imgs, plot_labels  = next(test_batches)\n'

In [0]:
'''
predictions = model.predict_generator(test_batches, steps = 1, workers = 1, verbose = 0)
'''

'\npredictions = model.predict_generator(test_batches, steps = 1, workers = 1, verbose = 0)\n'

In [0]:
'''
predictions = np.argmax(predictions,axis=1) + 1
'''

'\npredictions = np.argmax(predictions,axis=1) + 1\n'

In [0]:
'''
print(predictions)
'''

'\nprint(predictions)\n'

In [0]:
'''
plots(plot_imgs, figsize = (20,20), titles = predictions)
'''

'\nplots(plot_imgs, figsize = (20,20), titles = predictions)\n'